**요약**
- 사전학습된 OneFormer로부터 `test_image`에 대하여 예측을 합니다.
- 코드는 OneFormer [공식 데모 코드](https://colab.research.google.com/github/SHI-Labs/OneFormer/blob/main/colab/oneformer_colab.ipynb#scrollTo=xX8TLytOVal8)로부터 작성되었습니다.

<br>

**Inputs:**
- `dir_data`: 데이터가 있는 디렉토리
- `dir_save`: 예측 결과들이 저장될 디렉토리

<br>

**Outputs**:
- f`{dir_save}/ade/ade_din_pa_pa.pickle: Ade-20k dinat 모델의 panoptic task의 panoptic segmentation 결과
- f`{dir_save}/ade/ade_din_se_pa.pickle: Ade-20k dinat 모델의 semantic task의 panoptic segmentation 결과
- f`{dir_save}/ade/ade_swin_pa_pa.pickle: Ade-20k swin 모델의 panoptic task의 panoptic segmentation 결과
- f`{dir_save}/ade/ade_swin_se_pa.pickle: Ade-20k swin 모델의 semantic task의 panoptic segmentation 결과
- f`{dir_save}/co/co_din_pa_pa.pickle: CoCo dinat 모델의 panoptic task의 panoptic segmentation 결과
- f`{dir_save}/co/co_din_se_pa.pickle: CoCo dinat 모델의 semantic task의 panoptic segmentation 결과
- f`{dir_save}/co/co_swin_pa_pa.pickle: CoCo swin 모델의 panoptic task의 panoptic segmentation 결과
- f`{dir_save}/co/co_swin_sa_pa.pickle: CoCo swin 모델의 semantic task의 panoptic segmentation 결과
- f`{dir_save}/city/city_din_pa_pa.pickle: CoCo dinat 모델의 panoptic task의 panoptic segmentation 결과
- f`{dir_save}/city/city_din_se_pa.pickle: CoCo dinat 모델의 semantic task의 panoptic segmentation 결과
- f`{dir_save}/city/city_swin_pa_pa.pickle: CoCo swin 모델의 panoptic task의 panoptic segmentation 결과
- f`{dir_save}/city/city_swin_sa_pa.pickle: CoCo swin 모델의 semantic task의 panoptic segmentation 결과


In [1]:
dir_data = '../data'
dir_save = '../outputs'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WORLD_SIZE"] = "1"

import torch, gc
gc.collect()
torch.cuda.empty_cache()

print(torch.cuda.is_available())
print(torch.__version__)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())

%cd ./OneFormer/
import sys, os, distutils.core
dist = distutils.core.run_setup("./detectron2/setup.py")
sys.path.insert(0, os.path.abspath('./detectron2'))


/home/dongjin/miniconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
1.13.0
1
NVIDIA A100-SXM4-40GB MIG 7g.40gb
/home/dongjin/projects/da/notebooks/OneFormer


In [3]:
######
#@title 3. Import Libraries and other Utilities
######
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="oneformer")

# Import libraries
import numpy as np
import cv2
import torch
# from google.colab.patches import cv2_imshow
import imutils

# Import detectron2 utilities
from detectron2.config import get_cfg
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.data import MetadataCatalog
from demo.defaults import DefaultPredictor
from demo.visualizer import Visualizer, ColorMode


# import OneFormer Project
from oneformer import (
    add_oneformer_config,
    add_common_config,
    add_swin_config,
    add_dinat_config,
    add_convnext_config,
)

######
#@title 4. Define helper functions
######
cpu_device = torch.device("cpu")###############
gpu_device = torch.device("cuda")
SWIN_CFG_DICT = {"cityscapes": "configs/cityscapes/oneformer_swin_large_IN21k_384_bs16_90k.yaml",
            "coco": "configs/coco/oneformer_swin_large_IN21k_384_bs16_100ep.yaml",
            "ade20k": "configs/ade20k/oneformer_swin_large_IN21k_384_bs16_160k.yaml",}

DINAT_CFG_DICT = {"cityscapes": "configs/cityscapes/oneformer_dinat_large_bs16_90k.yaml",
            "coco": "configs/coco/oneformer_dinat_large_bs16_100ep.yaml",
            "ade20k": "configs/ade20k/oneformer_dinat_large_IN21k_384_bs16_160k.yaml",}

def setup_cfg(dataset, model_path, use_swin):
    # load config from file and command-line arguments
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_common_config(cfg)
    add_swin_config(cfg)
    add_dinat_config(cfg)
    add_convnext_config(cfg)
    add_oneformer_config(cfg)
    if use_swin:
        cfg_path = SWIN_CFG_DICT[dataset]
    else:
        cfg_path = DINAT_CFG_DICT[dataset]
    cfg.merge_from_file(cfg_path)
    cfg.MODEL.DEVICE = 'cuda'###############
    # cfg.MODEL.DEVICE = 'cuda'
    cfg.MODEL.WEIGHTS = model_path
    cfg.freeze()
    return cfg

def setup_modules(dataset, model_path, use_swin):
    cfg = setup_cfg(dataset, model_path, use_swin)
    predictor = DefaultPredictor(cfg)
    metadata = MetadataCatalog.get(
        cfg.DATASETS.TEST_PANOPTIC[0] if len(cfg.DATASETS.TEST_PANOPTIC) else "__unused"
    )
    if 'cityscapes_fine_sem_seg_val' in cfg.DATASETS.TEST_PANOPTIC[0]:
        from cityscapesscripts.helpers.labels import labels
        stuff_colors = [k.color for k in labels if k.trainId != 255]
        metadata = metadata.set(stuff_colors=stuff_colors)
    
    return predictor, metadata

# Ade20k Data Oneformer pretrained model로 부터 test_image inference pickle 파일 만들기

- ade_din_pa_pa.pickle
- ade_din_se_pa.pickle
- ade_swin_pa_pa.pickle
- ade_swin_se_pa.pickle

In [4]:
use_swin0 = False
predictor_ade_din, metadata_ade_din = setup_modules("ade20k", "250_16_dinat_l_oneformer_ade20k_160k.pth", use_swin0)

use_swin1 = True
predictor_ade_swin, metadata_ade_swin = setup_modules("ade20k", "250_16_swin_l_oneformer_ade20k_160k.pth", use_swin1)

os.makedirs(os.path.join('../', dir_save, 'ade'))

Loading config configs/ade20k/oneformer_dinat_large_IN21k_384_bs16_160k.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Loading config configs/ade20k/Base-ADE20K-UnifiedSegmentation.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[10/05 10:11:10 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from 250_16_dinat_l_oneformer_ade20k_160k.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  text_encoder.positional_embedding
  text_encoder.transformer.resblocks.0.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.0.attn.out_proj.{bias, weight}
  text_encoder.transformer.resblocks.0.ln_1.{bias, weight}
  text_encoder.transformer.resblocks.0.mlp.c_fc.{bias, weight}
  text_encoder.transformer.resblocks.0.mlp.c_proj.{bias, weight}
  text_encoder.transformer.resblocks.0.ln_2.{bias, weight}
  text_encoder.transformer.resblocks.1.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.1.attn.out_proj.{bias, weight}
  text_encoder.transformer.resblocks.1.ln_1.{bias, weight}
  text_encoder.transformer.resblocks.1.mlp.c_fc.{bias, weight}
  text_encoder.transformer.resblocks.1.mlp.c_proj.{bias, weight}
  text_encoder.transformer.resblocks.1.ln_2.{bias, weight}
  text_encoder.transformer.resblocks.2.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.2.

[10/05 10:11:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from 250_16_swin_l_oneformer_ade20k_160k.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  text_encoder.positional_embedding
  text_encoder.transformer.resblocks.0.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.0.attn.out_proj.{bias, weight}
  text_encoder.transformer.resblocks.0.ln_1.{bias, weight}
  text_encoder.transformer.resblocks.0.mlp.c_fc.{bias, weight}
  text_encoder.transformer.resblocks.0.mlp.c_proj.{bias, weight}
  text_encoder.transformer.resblocks.0.ln_2.{bias, weight}
  text_encoder.transformer.resblocks.1.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.1.attn.out_proj.{bias, weight}
  text_encoder.transformer.resblocks.1.ln_1.{bias, weight}
  text_encoder.transformer.resblocks.1.mlp.c_fc.{bias, weight}
  text_encoder.transformer.resblocks.1.mlp.c_proj.{bias, weight}
  text_encoder.transformer.resblocks.1.ln_2.{bias, weight}
  text_encoder.transformer.resblocks.2.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.2.

In [10]:
## ade_din_pa_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name = 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_ade_din(img, "panoptic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/ade/ade_din_pa_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:02<00:00,  1.98it/s]


In [7]:
## ade_din_se_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name = 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_ade_din(img, "semantic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)

import pickle
# save
with open(f'../{dir_save}/ade/ade_din_se_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

100%|██████████| 2194/2194 [13:23<00:00,  2.73it/s]


In [8]:
## ade_swin_pa_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name = 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_ade_swin(img, "panoptic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)

import pickle
# save
with open(f'../{dir_save}/ade/ade_swin_pa_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

100%|██████████| 2194/2194 [12:58<00:00,  2.82it/s]


In [9]:
## ade_swin_se_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name= 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_ade_swin(img, "semantic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)

import pickle
# save
with open(f'../{dir_save}/ade/ade_swin_se_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

100%|██████████| 2194/2194 [13:36<00:00,  2.69it/s]


# Coco Data Oneformer pretrained model로 부터 test_image inference pickle 파일 만들기

- co_din_pa_pa.pickle
- co_din_se_pa.pickle
- co_swin_pa_pa.pickle
- co_swin_se_pa.pickle

In [ ]:
use_swin0 = False
predictor_co_din, metadata_co_din = setup_modules("coco", "150_16_dinat_l_oneformer_coco_100ep.pth", use_swin0)

use_swin1 = True
predictor_co_swin, metadata_co_swin = setup_modules("coco", "150_16_swin_l_oneformer_coco_100ep.pth", use_swin1)

os.makedirs(os.path.join('../', dir_save, 'co'))

In [ ]:
## co_din_pa_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name= 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_co_din(img, "panoptic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/co/co_din_pa_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
## co_din_se_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name = 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_co_din(img, "semantic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/co/co_din_se_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
## co_swin_pa_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name = 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_co_swin(img, "panoptic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/co/co_swin_pa_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
## co_swin_se_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name = 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_co_swin(img, "semantic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/co/co_swin_se_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

# Cityscape Data Oneformer pretrained model로 부터 test_image inference pickle 파일 만들기

- city_din_pa_pa.pickle
- city_din_se_pa.pickle
- city_swin_pa_pa.pickle
- city_swin_se_pa.pickle

In [ ]:
use_swin0 = False
predictor_city_din, metadata_city_din = setup_modules("cityscapes", "250_16_dinat_l_oneformer_cityscapes_90k.pth", use_swin0)

use_swin1 = True
predictor_city_swin, metadata_city_swin = setup_modules("cityscapes", "250_16_swin_l_oneformer_cityscapes_90k.pth", use_swin1)

os.makedirs(os.path.join('../', dir_save, 'city'))

In [ ]:
## city_din_pa_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name= 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_city_din(img, "panoptic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/city/city_din_pa_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
## city_din_se_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name= 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_city_din(img, "semantic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/city/city_din_se_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
## city_swin_pa_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name= 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_city_swin(img, "panoptic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/city/city_swin_pa_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
## city_swin_se_pa.picle 만들기
import cv2
import numpy as np
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('detectron2').setLevel(logging.ERROR) # 이상한 문구 뜨는거 끄는 것.

result={}
for ind in tqdm(range(1898)): 
    name= 'TEST_' + str(ind).zfill(4) + '.png'
    test_img_path = os.path.join('../', dir_data, 'test_image', name)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out1,out2 = predictor_city_swin(img, "semantic")['panoptic_seg']
    result[ind]=(out1.cpu().numpy().astype(np.uint8),out2)


import pickle
# save
with open(f'../{dir_save}/city/city_swin_se_pa.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)